## Building a Conversational Assistant with Function Calling

* **Created by:** Eric Martinez
* **For:** CSCI 3351
* **At:** University of Texas Rio-Grande Valley

## Setup

In [2]:
%pip install -U --quiet pydantic openai gradio geocoder

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vcrpy 5.1.0 requires urllib3<2; python_version < "3.10", but you have urllib3 2.2.1 which is incompatible.
kubernetes 28.1.0 requires urllib3<2.0,>=1.24.2, but you have urllib3 2.2.1 which is incompatible.
You should consider upgrading via the 'c:\users\kty325\.pyenv\pyenv-win\versions\3.9.6\python.exe -m pip install --upgrade pip' command.


## Define Tools

What are some helpful functions that we might want to make available to an assistant?

In [18]:
def sum(a:int, b:int=1):
    "Adds a + b"
    return a + b

print(sum(a=2, b=3))

5


In [19]:
import requests

def geocode(city_name:str):
    "Geocodes a city name into latitude and longitude data"
    url = f'https://geocoding-api.open-meteo.com/v1/search?name={city_name}&count=10&language=en&format=json'

    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')  
    else:
        return response.json() # If successful, return json response
    
print(geocode(city_name="Edinburg")['results'][0])

{'id': 4688275, 'name': 'Edinburg', 'latitude': 26.30174, 'longitude': -98.16334, 'elevation': 30.0, 'feature_code': 'PPLA2', 'country_code': 'US', 'admin1_id': 4736286, 'admin2_id': 4697444, 'timezone': 'America/Chicago', 'population': 84497, 'postcodes': ['78539', '78540'], 'country_id': 6252001, 'country': 'United States', 'admin1': 'Texas', 'admin2': 'Hidalgo'}


In [20]:
def weather(latitude:float, longitude:float):
    "Returns the weather conditions for a given latitude and longitude"
    url = f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,is_day,precipitation,rain,showers,snowfall&timezone=America%2FChicago'

    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')  
    else:
        return response.json() # If successful, return json response
    
print(weather(latitude=26.30174, longitude=-98.16334))

{'latitude': 26.309208, 'longitude': -98.153625, 'generationtime_ms': 0.5550384521484375, 'utc_offset_seconds': -18000, 'timezone': 'America/Chicago', 'timezone_abbreviation': 'CDT', 'elevation': 30.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C', 'is_day': '', 'precipitation': 'mm', 'rain': 'mm', 'showers': 'mm', 'snowfall': 'cm'}, 'current': {'time': '2024-04-22T15:45', 'interval': 900, 'temperature_2m': 19.5, 'is_day': 1, 'precipitation': 0.0, 'rain': 0.0, 'showers': 0.0, 'snowfall': 0.0}}


In [21]:
from datetime import datetime
from datetime import timedelta

# gets todays date
def get_todays_date():
    "Gets today's date"
    return datetime.now().strftime('%B %d, %Y')

# Example:
print(get_todays_date())

April 22, 2024


In [22]:
# gets tomorrows date
def get_tomorrows_date():
    "Gets tomorrow's date"
    now = datetime.now()
    tomorrow = now + timedelta(days=1)
    return tomorrow.strftime('%B %d, %Y')

print(get_tomorrows_date())

April 23, 2024


In [23]:
# gets the current time
def get_current_time():
    "Gets the current time"
    return datetime.now().strftime("%m/%d/%Y %I:%M %p")

# Example:
print(get_current_time())

04/22/2024 03:45 PM


In [24]:
import geocoder

# finds your city and state based on your IP address
def user_location():
    "Guesses the users current location in the format of <city>, <state> based on IP address. But should always ask the user when scheduling."
    g = geocoder.ip('me')
    city = g.city
    state = g.state
    return f"{city}, {state}"

# Example:
print(user_location())

Edinburg, Texas


## Format for OpenAI

In [25]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

def schema(f):
    kw = {n: (o.annotation, ... if o.default == Parameter.empty else o.default)
          for n, o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).model_json_schema()
    function_schema = dict(name=f.__name__, description=f.__doc__, parameters=s)
    return {
        "type": "function",
        "function": function_schema
    }

def call_func(name, arguments):
    f = globals()[name]
    return f(**json.loads(arguments))

In [26]:
schema(sum)

{'type': 'function',
 'function': {'name': 'sum',
  'description': 'Adds a + b',
  'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
    'b': {'default': 1, 'title': 'B', 'type': 'integer'}},
   'required': ['a'],
   'title': 'Input for `sum`',
   'type': 'object'}}}

In [27]:
call_func("sum", '{"a": 2, "b": 3}')

5

## New Chat Completion Function

In [28]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)  # take environment variables from .env.

from openai import OpenAI

client = OpenAI(
    base_url=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY")
)

def chat_completion(
    message,
    model="gpt-4",
    prompt="You are a helpful assistant.",
    temperature=0,
    messages=[],
    tools=[]
):
    # Add the prompt to the messages list
    if prompt is not None:
        messages = [{"role": "system", "content": prompt}] + messages

    if message is not None:
        # Add the user's message to the messages list
        messages += [{"role": "user", "content": message}]

    # NEW
    if tools:
        tool_schemas = [schema(f) for f in tools]
    else:
        tool_schemas = None
        
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    print(f"messages: {messages}")
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        tools=tool_schemas
    )
    
    # Updated
    assistant_message = completion.choices[0].message
        
    return assistant_message

Test it out

In [29]:
chat_completion("Add 2+3", model="gpt-4", tools=[sum])

messages: [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Add 2+3'}]


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_bsVZoKLLYaZBUlp2h6jr7c41', function=Function(arguments='{\n  "a": 2,\n  "b": 3\n}', name='sum'), type='function')])

## Building a Function Calling App

In [30]:
# Define a function to handle the chat interaction with the AI model

def gradio_assistant_chat(prompt, message, chatbot_messages, history_state, allowed_tools=None):
    # Initialize chatbot_messages and history_state if they are not provided
    chatbot_messages = chatbot_messages or []
    history_state = history_state or []
    
    
    # Try to get the AI's reply using the chat_completion
    try:
        assistant_message = chat_completion(message, model="gpt-4", prompt=prompt, messages=history_state, tools=allowed_tools)
    except Exception as e:
        # If an error occurs, raise a Gradio error
        raise gr.Error(e)
        
    # UPDATED
    history_state.append({"role": "user", "content": message})

    while(assistant_message.tool_calls):
        for tool_call in assistant_message.tool_calls:
            tool_call_json = json.dumps({ "name": tool_call.function.name, "arguments": tool_call.function.arguments})
#             print(f"Tool Call: {tool_call_json}")
            history_state.append({"role": assistant_message.role, "content": tool_call_json})
            
        for tool_call in assistant_message.tool_calls:
            results = call_func(tool_call.function.name, tool_call.function.arguments)
            results_str = str(results) # convert to string
#             print(f"Result: {results_str}")
            history_state.append({"role": "function", "tool_call_id": tool_call.id, "name": tool_call.function.name, "content": results_str})

        # Try to get the AI's reply using the get_ai_reply function
        try:
            assistant_message = chat_completion(None, model="gpt-4", prompt=prompt, messages=history_state, tools=allowed_tools)
        except Exception as e:
            # If an error occurs, raise a Gradio error
            raise gr.Error(e)
        
    # Append the user's message and the AI's reply to the chatbot_messages list
    if(assistant_message.content):
        chatbot_messages.append((message, assistant_message.content.strip()))
    else:
        chatbot_messages.append((message, None))
    
    # Append the user's message and the AI's reply to the history_state list
    history_state.append({"role": "assistant", "content": assistant_message.content})
    
    # Return None (empty out the user's message textbox), the updated chatbot_messages, and the updated history_state
    return None, chatbot_messages, history_state

In [33]:
def chat(message, chatbot_messages, history_state):
    prompt = """
    You are a helpful AI assistant named Jarvis.

    Today's date: {todays_date}
    Current time: {current_time}
    User's location: {user_location}

    # Tools

    You have access to many tools.

    When you are handed the result of a tool call think critically about the user's original question that triggered you to want to invoke a tool.

    Critically consider if the user's question has been answered or what additional tool calls might be needed to answer the question.    
    """.format(
        todays_date=get_todays_date(),
        current_time=get_current_time(),
        user_location=user_location()
    )

    allowed_tools = [sum, weather, get_tomorrows_date]
    return gradio_assistant_chat(prompt, message, chatbot_messages, history_state, allowed_tools=allowed_tools)


In [34]:
import gradio as gr


# Define a function to return a chatbot app using Gradio
def get_chatbot_app(share=False):
    # Create the Gradio interface using the Blocks layout
    with gr.Blocks() as app:
        with gr.Tab("Conversation"):
            with gr.Row():
                with gr.Column():
                    # Create a chatbot interface for the conversation
                    chatbot = gr.Chatbot(label="Conversation")
                    # Create a textbox for the user's message
                    message = gr.Textbox(label="Message")
                    # Create a state object to store the conversation history
                    history_state = gr.State()
                    # Create a state object to store the available tools
                    tools = gr.State()
                    # Create a button to send the user's message
                    btn = gr.Button(value="Send")
                btn.click(chat, inputs=[message, chatbot, history_state], outputs=[message, chatbot, history_state])
        # Return the app
        return app
    
# Call the launch_chatbot function to start the chatbot interface using Gradio
# Set the share parameter to False, meaning the interface will not be publicly accessible
get_chatbot_app().launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


messages: [{'role': 'system', 'content': "\n    You are a helpful AI assistant named Jarvis.\n\n    Today's date: April 22, 2024\n    Current time: 04/22/2024 03:46 PM\n    User's location: Edinburg, Texas\n\n    # Tools\n\n    You have access to many tools.\n\n    When you are handed the result of a tool call think critically about the user's original question that triggered you to want to invoke a tool.\n\n    Critically consider if the user's question has been answered or what additional tool calls might be needed to answer the question.    \n    "}, {'role': 'user', 'content': 'Do I need an  umbrella today?'}]
messages: [{'role': 'system', 'content': "\n    You are a helpful AI assistant named Jarvis.\n\n    Today's date: April 22, 2024\n    Current time: 04/22/2024 03:46 PM\n    User's location: Edinburg, Texas\n\n    # Tools\n\n    You have access to many tools.\n\n    When you are handed the result of a tool call think critically about the user's original question that triggered

## Experimental: Code Interpreter

In [35]:
import ast

def python(code:str):
    "Return result of executing `code` using python."
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None
    
    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgts = [ast.Name(id='_result', ctx=ast.Store())]
        assign = ast.Assign(targets=tgts, value=last_node.value)
        tree.body[-1] = ast.fix_missing_locations(assign)

    ns = {}
    exec(compile(tree, filename='<ast>', mode='exec'), ns)
    return ns.get('_result', None)

print(python("print('hello world')"))

hello world
None


In [38]:
# Define a function to handle the chat interaction with the AI model

def extract_python_info(history_state):
    # Initialize variables to store the last Python call information
    last_python_call_arguments = None
    last_python_call_result = None

    # Iterate through the messages
    for i in range(1, len(history_state)):
        if history_state[i]['role'] == 'function' and history_state[i]['name'] == 'python':
            # Get the function name and result from the current message
            function_name = history_state[i]['name']
            result = history_state[i]['content']

            # Get the arguments from the previous message
            tool_call = json.loads(history_state[i-1]['content'])
            

            last_python_call_arguments = tool_call['arguments']
            last_python_call_result = result
            
    
    if(last_python_call_arguments):
        last_python_call_arguments = json.loads(last_python_call_arguments)
    
        if("code" in last_python_call_arguments):
            last_python_call_arguments = last_python_call_arguments["code"]
        
    return last_python_call_arguments, last_python_call_result

def format_as_markdown(code_block, language="python"):
    md = f"```{language}\n"
    if(code_block):
        md += code_block
    md += f"\n```"
    return md
    
def chat(message, chatbot_messages, history_state):
    prompt = """
    You are a helpful AI assistant named Jarvis.

    Today's date: {todays_date}
    Current time: {current_time}
    User's location: {user_location}
    
    # Tools
    
    You have access to many tools.
    
    When you are handed the result of a tool call think critically about the user's original question that triggered you to want to invoke a tool.
    
    Critically consider if the user's question has been answered or what additional tool calls might be needed to answer the question.
    
    Make sure you use tool_calls in the way you were fine-tuned to use.
    
    Use the python tool when you need to perform computation.
    
    # Conversions
    
    Make sure to use the python tool to convert units to imperial if its appropriate based on the user's location
    
    # Useful APIs to use within python tool
    
    # SerpAPI
    
    Get the SerpAPI API key from `os.getenv('SERPAPI_KEY')
    
    ## Google Search
    
    Example: GET https://serpapi.com/search.json?q=Coffee&location=Austin,+Texas,+United+States&hl=en&gl=us&google_domain=google.com&api_key=<api_key>

    Results are in the "organic_results" key. Limit to just the first 10 results.
    
    ## Google News
    
    Example: GET https://serpapi.com/search.json?engine=google_news&q=pizza&api_key=<api_key>

    Results are in the "news_results" key. Limit to just the first 10 results.

    """.format(
        todays_date=get_todays_date(),
        current_time=get_current_time(),
        user_location=user_location()
    )
    
    # Initialize chatbot_messages and history_state if they are not provided
    allowed_tools = [weather, get_tomorrows_date, python]
    message, chatbot_messages, history_state =  gradio_assistant_chat(prompt, message, chatbot_messages, history_state, allowed_tools=allowed_tools)
    
    code, result = extract_python_info(history_state)
    
    code = format_as_markdown(code, language="python")
    result = format_as_markdown(result, language="python")
            
    return message, chatbot_messages, history_state, code, result

In [39]:
import gradio as gr


# Define a function to return a chatbot app using Gradio
def get_chatbot_app(share=False):
    # Create the Gradio interface using the Blocks layout
    with gr.Blocks() as app:
        with gr.Tab("Conversation"):
            with gr.Row():
                with gr.Column():
                    # Create a chatbot interface for the conversation
                    chatbot = gr.Chatbot(label="Conversation")
                    # Create a textbox for the user's message
                    message = gr.Textbox(label="Message")
                    # Create a state object to store the conversation history
                    history_state = gr.State()
                    # Create a state object to store the available tools
                    tools = gr.State()
                    # Create a button to send the user's message
                    btn = gr.Button(value="Send")
        with gr.Tab("Debug - Code Interpreter"):
            with gr.Row():
                with gr.Column():
                    code = gr.Markdown(label="Last Executed", value="```python\n# nothing yet\n```")
                with gr.Column():
                    result = gr.Markdown(label="Last Result", value="```python\n# nothing yet\n```")
        # Return the app
        btn.click(chat, inputs=[message, chatbot, history_state], outputs=[message, chatbot, history_state, code, result])
        return app
    
# Call the launch_chatbot function to start the chatbot interface using Gradio
# Set the share parameter to False, meaning the interface will not be publicly accessible
get_chatbot_app().launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


messages: [{'role': 'system', 'content': '\n    You are a helpful AI assistant named Jarvis.\n\n    Today\'s date: April 22, 2024\n    Current time: 04/22/2024 03:57 PM\n    User\'s location: Edinburg, Texas\n    \n    # Tools\n    \n    You have access to many tools.\n    \n    When you are handed the result of a tool call think critically about the user\'s original question that triggered you to want to invoke a tool.\n    \n    Critically consider if the user\'s question has been answered or what additional tool calls might be needed to answer the question.\n    \n    Make sure you use tool_calls in the way you were fine-tuned to use.\n    \n    Use the python tool when you need to perform computation.\n    \n    # Conversions\n    \n    Make sure to use the python tool to convert units to imperial if its appropriate based on the user\'s location\n    \n    # Useful APIs to use within python tool\n    \n    # SerpAPI\n    \n    Get the SerpAPI API key from `os.getenv(\'SERPAPI_KEY\'

messages: [{'role': 'system', 'content': '\n    You are a helpful AI assistant named Jarvis.\n\n    Today\'s date: April 22, 2024\n    Current time: 04/22/2024 04:00 PM\n    User\'s location: Edinburg, Texas\n    \n    # Tools\n    \n    You have access to many tools.\n    \n    When you are handed the result of a tool call think critically about the user\'s original question that triggered you to want to invoke a tool.\n    \n    Critically consider if the user\'s question has been answered or what additional tool calls might be needed to answer the question.\n    \n    Make sure you use tool_calls in the way you were fine-tuned to use.\n    \n    Use the python tool when you need to perform computation.\n    \n    # Conversions\n    \n    Make sure to use the python tool to convert units to imperial if its appropriate based on the user\'s location\n    \n    # Useful APIs to use within python tool\n    \n    # SerpAPI\n    \n    Get the SerpAPI API key from `os.getenv(\'SERPAPI_KEY\'

messages: [{'role': 'system', 'content': '\n    You are a helpful AI assistant named Jarvis.\n\n    Today\'s date: April 22, 2024\n    Current time: 04/22/2024 04:00 PM\n    User\'s location: Edinburg, Texas\n    \n    # Tools\n    \n    You have access to many tools.\n    \n    When you are handed the result of a tool call think critically about the user\'s original question that triggered you to want to invoke a tool.\n    \n    Critically consider if the user\'s question has been answered or what additional tool calls might be needed to answer the question.\n    \n    Make sure you use tool_calls in the way you were fine-tuned to use.\n    \n    Use the python tool when you need to perform computation.\n    \n    # Conversions\n    \n    Make sure to use the python tool to convert units to imperial if its appropriate based on the user\'s location\n    \n    # Useful APIs to use within python tool\n    \n    # SerpAPI\n    \n    Get the SerpAPI API key from `os.getenv(\'SERPAPI_KEY\'

messages: [{'role': 'system', 'content': '\n    You are a helpful AI assistant named Jarvis.\n\n    Today\'s date: April 22, 2024\n    Current time: 04/22/2024 04:00 PM\n    User\'s location: Edinburg, Texas\n    \n    # Tools\n    \n    You have access to many tools.\n    \n    When you are handed the result of a tool call think critically about the user\'s original question that triggered you to want to invoke a tool.\n    \n    Critically consider if the user\'s question has been answered or what additional tool calls might be needed to answer the question.\n    \n    Make sure you use tool_calls in the way you were fine-tuned to use.\n    \n    Use the python tool when you need to perform computation.\n    \n    # Conversions\n    \n    Make sure to use the python tool to convert units to imperial if its appropriate based on the user\'s location\n    \n    # Useful APIs to use within python tool\n    \n    # SerpAPI\n    \n    Get the SerpAPI API key from `os.getenv(\'SERPAPI_KEY\'

messages: [{'role': 'system', 'content': '\n    You are a helpful AI assistant named Jarvis.\n\n    Today\'s date: April 22, 2024\n    Current time: 04/22/2024 04:11 PM\n    User\'s location: Edinburg, Texas\n    \n    # Tools\n    \n    You have access to many tools.\n    \n    When you are handed the result of a tool call think critically about the user\'s original question that triggered you to want to invoke a tool.\n    \n    Critically consider if the user\'s question has been answered or what additional tool calls might be needed to answer the question.\n    \n    Make sure you use tool_calls in the way you were fine-tuned to use.\n    \n    Use the python tool when you need to perform computation.\n    \n    # Conversions\n    \n    Make sure to use the python tool to convert units to imperial if its appropriate based on the user\'s location\n    \n    # Useful APIs to use within python tool\n    \n    # SerpAPI\n    \n    Get the SerpAPI API key from `os.getenv(\'SERPAPI_KEY\'

messages: [{'role': 'system', 'content': '\n    You are a helpful AI assistant named Jarvis.\n\n    Today\'s date: April 22, 2024\n    Current time: 04/22/2024 04:13 PM\n    User\'s location: Edinburg, Texas\n    \n    # Tools\n    \n    You have access to many tools.\n    \n    When you are handed the result of a tool call think critically about the user\'s original question that triggered you to want to invoke a tool.\n    \n    Critically consider if the user\'s question has been answered or what additional tool calls might be needed to answer the question.\n    \n    Make sure you use tool_calls in the way you were fine-tuned to use.\n    \n    Use the python tool when you need to perform computation.\n    \n    # Conversions\n    \n    Make sure to use the python tool to convert units to imperial if its appropriate based on the user\'s location\n    \n    # Useful APIs to use within python tool\n    \n    # SerpAPI\n    \n    Get the SerpAPI API key from `os.getenv(\'SERPAPI_KEY\'

messages: [{'role': 'system', 'content': '\n    You are a helpful AI assistant named Jarvis.\n\n    Today\'s date: April 22, 2024\n    Current time: 04/22/2024 04:14 PM\n    User\'s location: Edinburg, Texas\n    \n    # Tools\n    \n    You have access to many tools.\n    \n    When you are handed the result of a tool call think critically about the user\'s original question that triggered you to want to invoke a tool.\n    \n    Critically consider if the user\'s question has been answered or what additional tool calls might be needed to answer the question.\n    \n    Make sure you use tool_calls in the way you were fine-tuned to use.\n    \n    Use the python tool when you need to perform computation.\n    \n    # Conversions\n    \n    Make sure to use the python tool to convert units to imperial if its appropriate based on the user\'s location\n    \n    # Useful APIs to use within python tool\n    \n    # SerpAPI\n    \n    Get the SerpAPI API key from `os.getenv(\'SERPAPI_KEY\'

messages: [{'role': 'system', 'content': '\n    You are a helpful AI assistant named Jarvis.\n\n    Today\'s date: April 22, 2024\n    Current time: 04/22/2024 04:15 PM\n    User\'s location: Edinburg, Texas\n    \n    # Tools\n    \n    You have access to many tools.\n    \n    When you are handed the result of a tool call think critically about the user\'s original question that triggered you to want to invoke a tool.\n    \n    Critically consider if the user\'s question has been answered or what additional tool calls might be needed to answer the question.\n    \n    Make sure you use tool_calls in the way you were fine-tuned to use.\n    \n    Use the python tool when you need to perform computation.\n    \n    # Conversions\n    \n    Make sure to use the python tool to convert units to imperial if its appropriate based on the user\'s location\n    \n    # Useful APIs to use within python tool\n    \n    # SerpAPI\n    \n    Get the SerpAPI API key from `os.getenv(\'SERPAPI_KEY\'

Traceback (most recent call last):
  File "c:\users\kty325\.pyenv\pyenv-win\versions\3.9.6\lib\site-packages\gradio\queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "c:\users\kty325\.pyenv\pyenv-win\versions\3.9.6\lib\site-packages\gradio\route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\users\kty325\.pyenv\pyenv-win\versions\3.9.6\lib\site-packages\gradio\blocks.py", line 1788, in process_api
    result = await self.call_function(
  File "c:\users\kty325\.pyenv\pyenv-win\versions\3.9.6\lib\site-packages\gradio\blocks.py", line 1340, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\users\kty325\.pyenv\pyenv-win\versions\3.9.6\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\users\kty325\.pyenv\pyenv-win\versions\3.9.6\lib\site-packages\anyio\_backends\_asyncio.py", line 87